In [1]:
from IPython.display import display

from sympy import *

from monom import *

In [2]:
t, x, y, tau, h, Re, p0 = symbols(r't, x, y, tau, h, Re, p_0', real=True)
u, v, p = (f(t, x, y) for f in symbols('u, v, p', cls=Function))

In [3]:
pda_f, pda_v = (u, v, p), (t, x, y)
pda_n, pda_clp = 5, 3
pda_p = Rational(0, 1), Rational(0, 1), Rational(0, 1)

In [4]:
Monom.variables = 1 + len(pda_v)
Monom.cmp = Monom.TOPdeglex
Monom.zero = Monom(0 for v in range(Monom.variables))
pda_fun = dict(zip(pda_f,\
             (Monom(0 if v else i for v in range(Monom.variables))\
              for i in range(1, Monom.variables))))
pda_var = dict(zip(pda_v,\
             (Monom(0 if v != i else 1 for v in range(Monom.variables))\
              for i in range(1, Monom.variables))))

In [5]:
def T(f, i1, j1, k1):
    return sum(sum(sum(\
        diff(f, t, i, x, j, y, k)*(tau*(i1+pda_p[0]))**i \
                                   *(h*(j1+pda_p[1]))**j \
                                   *(h*(k1+pda_p[2]))**k/\
                 (factorial(i)*factorial(j)*factorial(k))\
        for i in range(pda_n-j-k))\
        for k in range(pda_n-j))  \
        for j in range(pda_n))

In [6]:
expand((T(u, 0, 0, 1)-T(u, 0, 0, -1))/(2*h))

h**2*Derivative(u(t, x, y), (y, 3))/6 + Derivative(u(t, x, y), y)

In [7]:
def clip(f):
    f = f.expand()
    return [f.coeff(tau, 0).coeff(h, 0),\
            f.coeff(h, 0).coeff(tau, 1),\
            f.coeff(tau, 0).coeff(h, 2)]

def df2m(a):
    assert a.func == Derivative
    m = pda_fun[a.args[0]]
    for xi in a.args[1:]:
        if isinstance(xi, Symbol):
            m = m*pda_var[xi]
        else:
            m = m*pda_var[xi[0]]**xi[1]
    return m

def m2df(m):
    r = pda_f[m[0]-1]
    for i in range(1, len(m)):
        r = r.diff(pda_v[i-1], m[i])
    return r

def findDiv(a, d):
    r = None
    def find(a, r):
        if a.args:
            if a.func == Derivative and a.args[0] in pda_fun:
                m = df2m(a)
                if m.divisible(d) and (not r or m.cmp(r) > 0):
                    r = m
            else:
                for s in a.args:
                    r = find(s, r)
        return r
    return find(a, r)

def reduction(f1, f2, m, c, shift):
    assert shift < pda_clp
    r = [f1[i] for i in range(shift)]
    if not m:
        for i in range(shift, pda_clp):
            r.append(expand(f1[i] - f2[i-shift]*c))
    else:
        for i in range(shift, pda_clp):
            r.append(expand(f1[i] - f2[i-shift].diff(*m)*c))
    return r

def NF(f, df, G, head=False, trace=False):
    assert len(df) == len(G)
#     print(df2m(df[0]))
    ms = [df2m(d) for d in df]
    for i in range(0 if head else 1, pda_clp):
        t = 0
        if f[i]:
            while True:
                r = None
                for l in range(len(ms)):
                    r = findDiv(f[i], ms[l])
                    if r:
                        break
                if not r: 
                    break
                c, deg, m = 0, 7, m2df(r)
                while c == 0:
                    c = f[i].coeff(m, deg)
                    deg -= 1
                    assert deg >= 0
#                 print(c, m, deg+1)
                if deg:
                    c *= m**deg
                m = r/ms[l]
                d = []
                for k in range(len(pda_v)):
                    if m[k+1] > 0:
                        d.append(pda_v[k])
                        if m[k+1] > 1:
                            d.append(m[k+1])
                if trace:
                    print(">"*12)
                    eq = Symbol("eq%d" % (l+1), real=True)
                    if d:
                        display(Derivative(*tuple([eq] + d))*c*h**i)
                    else:
                        display(eq*c*h**i)
                f = reduction(f, G[l], tuple(d), c/G[l][0].coeff(df[l]), i)
                if trace:
                    print("res =")
                    display(f)
                    print("<"*12)
                t += 1
#                 if t > 6: break
    return f

def compact(f):
    def cmpct(a):
        if not a.args:
            return a
        else:
            if a in pda_fun:
                return Symbol("%s" % a.func, real=True)
            elif a.func == Derivative and a.args[0] in pda_fun:
                m = []
                for xi in a.args[1:]:
                    if isinstance(xi, Symbol):
                        m.append(str(xi))
                    else:
                        m.append(str(xi[0])*xi[1])
                return Symbol("%s_{%s}" % (a.args[0].func, "".join(m)), real=True)
            else:
                return a.func(*tuple(cmpct(s) for s in a.args))
    return cmpct(f)

def prn(a):
    display(compact(a[0]))
    print("tau =>")
    display(compact(a[1]))
    print("h^2 =>")
    display(compact(a[2]))
    
def prnlatex(a):
    print(latex(compact(a[0])))
    print(r"+\tau\big(")
    print(latex(compact(a[1]))) 
    print(r"\big)")
    print(r"+h^2\big(")
    print(latex(compact(a[2]))) 
    print(r"\big)")    

In [8]:
def Dt(a):
    return (T(a, 1, 0, 0) - T(a, 0, 0, 0))/(tau)
def Dx(a):
    return (T(a, 0, 1, 0) - T(a, 0, -1, 0))/(2*h)
def Dy(a):
    return (T(a, 0, 0, 1) - T(a, 0, 0, -1))/(2*h)

def DD(a):
    return (T(a, 0, 1, 0) + T(a, 0, 0, 1) +\
                -4*T(a, 0, 0, 0) +\
           T(a, 0, -1, 0) + T(a, 0, 0, -1))/h**2

def DD2(a):
    return (T(a, 0, 2, 0) + T(a, 0, 0, 2) +\
                -4*T(a, 0, 0, 0) +\
           T(a, 0, -2, 0) + T(a, 0, 0, -2))/(4*h**2)

**Taylor decaying**

This is a classical Navier-Stokes problem, in general used to state the con-
vergence order of the considered formulae. Exact solution is
\begin{equation} \label{eq1}
\left\lbrace 
\begin{array}{l}
 u = -e^{-2t/\mathrm{Re}} \cos(x) \sin(y) ,\\
 v =  e^{-2t/\mathrm{Re}} \sin(x) \cos(y) ,\\
 p = -e^{-4t/\mathrm{Re}} (\cos(2x) + \cos(2y))/4 ,
\end{array}
\right. 
\end{equation} 

In [9]:
TD = {u: -exp(-2*t/Re)*cos(x)*sin(y),\
      v:  exp(-2*t/Re)*sin(x)*cos(y),\
      p: -exp(-4*t/Re)*(cos(2*x)+cos(2*y))/4}

**Kovasznay flow**

This is a classical Navier-Stokes problem, in general used to state the con-
vergence order of the considered formulae. Exact solution is
\begin{equation} \label{eq2}
\left\lbrace 
\begin{array}{l}
 u = 1-e^{\lambda x} \cos(2 \pi y) ,\\
 v = \lambda/{2\pi} e^{\lambda x} \sin(2 \pi y) ,\\
 p = p_0-\frac{1}{2}e^{2\lambda x} ,
\end{array}
\right. 
\end{equation}
where $\lambda = \mathrm{Re}/2-\sqrt{\mathrm{Re}^2/4+4\pi^2}$.

In [10]:
Lambda = Re/2 - sqrt(Re**2/4 + 4*pi**2)
KF = {u: 1 - exp(Lambda*x)*cos(2*pi*y),\
      v: Lambda/(2*pi)*exp(Lambda*x)*sin(2*pi*y),\
      p: p0 - exp(2*Lambda*x)/2}

# FDA2

Hans Johnston and Jian-Guo Liu, Finite Difference Schemes for Incompressible Flow Based on Local Pressure Boundary Conditions. Journal of Computational Physics 180, 120–154 (2002) doi:10.1006/jcph.2002.7079

page 126

\begin{equation}
\left\lbrace 
\begin{array}{rl}
F^1:& \operatorname{D_1}(u) + \operatorname{D_2}(v)=0 ,\\[4pt]
F^2:& \operatorname{D_t}(u) + u\operatorname{D_1}(u)+
v\operatorname{D_2}(u) + 
\operatorname{D_1}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (u)
=0 ,\\[4pt]
F^3:& \operatorname{D_t}(v) + u\operatorname{D_1}(v)+
v\operatorname{D_2}(v) +
\operatorname{D_2}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (v)
=0 ,\\[4pt]
F^4:& \tilde{\Delta}(p) 
- 2\operatorname{D_1}(u)\operatorname{D_2}(v)
+ 2\operatorname{D_1}(v)\operatorname{D_2}(u)
=0 .
\end{array}
\right. 
\end{equation}

In [11]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + u*Dx(u) + v*Dy(u) + Dx(p) - DD(u)/Re)
F3 = clip(Dt(v) + u*Dx(v) + v*Dy(v) + Dy(p) - DD(v)/Re)
F4 = clip(DD(p) - 2*Dx(u)*Dy(v) + 2*Dx(v)*Dy(u))

In [12]:
prn(F1);print("-"*25)
f1 = NF(F1, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F1, F2, F3, F4], head=False)
prn(f1)
prnlatex(f1)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


-Re*p_{yy}/6 - Re*u*v_{xy}/6 - Re*u_{y}*v_{x}/6 - Re*v*v_{yy}/6 - Re*v_{ty}/6 - Re*v_{y}**2/6 + v_{yyy}/3

u_{x} + v_{y}
+\tau\big(
0
\big)
+h^2\big(
- \frac{Re p_{yy}}{6} - \frac{Re u v_{xy}}{6} - \frac{Re u_{y} v_{x}}{6} - \frac{Re v v_{yy}}{6} - \frac{Re v_{ty}}{6} - \frac{Re v_{y}^{2}}{6} + \frac{v_{yyy}}{3}
\big)


In [13]:
prn([f.subs(TD).doit().simplify() for f in f1])

0

tau =>


0

h^2 =>


0

In [14]:
prn([f.subs(KF).doit().simplify() for f in f1])

0

tau =>


0

h^2 =>


-(Re**3 - Re**2*sqrt(Re**2 + 16*pi**2) + 16*pi**2*Re - 8*pi**2*sqrt(Re**2 + 16*pi**2))*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2)*cos(2*pi*y)/12

In [15]:
prn(F2);print("-"*25)
f2 = NF(F2, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F1, F2, F3, F4], head=False)
prn(f2)
prnlatex(f2)

p_{x} + u*u_{x} + u_{t} + u_{y}*v - u_{xx}/Re - u_{yy}/Re

tau =>


u_{tt}/2

h^2 =>


p_{xxx}/6 + u*u_{xxx}/6 + u_{yyy}*v/6 - u_{xxxx}/(12*Re) - u_{yyyy}/(12*Re)

...

tau =>


u_{tt}/2

h^2 =>


Re**2*p_{x}*v**2/12 - Re**2*u*v**2*v_{y}/12 + Re**2*u_{t}*v**2/12 + Re**2*u_{y}*v**3/12 - Re*p_{tx}/12 + Re*p_{xy}*v/12 - Re*p_{x}*v_{y}/12 - Re*p_{yy}*u/12 - Re*p_{y}*u_{y}/4 - Re*u**2*v_{xy}/12 - Re*u*u_{y}*v_{x}/3 - Re*u*v*v_{yy}/6 - Re*u_{tt}/12 - Re*u_{y}*v*v_{y}/3 - Re*u_{y}*v_{t}/3 + Re*v**2*v_{xy}/12 - p_{xyy}/6 + u*v_{yyy}/6 + u_{y}*v_{yy}/3 + v*v_{xyy}/6 - 2*v_{xy}*v_{y}/3 + v_{x}*v_{yy}/3 - v_{xyyy}/(6*Re)

p_{x} + u u_{x} + u_{t} + u_{y} v - \frac{u_{xx}}{Re} - \frac{u_{yy}}{Re}
+\tau\big(
\frac{u_{tt}}{2}
\big)
+h^2\big(
\frac{Re^{2} p_{x} v^{2}}{12} - \frac{Re^{2} u v^{2} v_{y}}{12} + \frac{Re^{2} u_{t} v^{2}}{12} + \frac{Re^{2} u_{y} v^{3}}{12} - \frac{Re p_{tx}}{12} + \frac{Re p_{xy} v}{12} - \frac{Re p_{x} v_{y}}{12} - \frac{Re p_{yy} u}{12} - \frac{Re p_{y} u_{y}}{4} - \frac{Re u^{2} v_{xy}}{12} - \frac{Re u u_{y} v_{x}}{3} - \frac{Re u v v_{yy}}{6} - \frac{Re u_{tt}}{12} - \frac{Re u_{y} v v_{y}}{3} - \frac{Re u_{y} v_{t}}{3} + \frac{Re v^{2} v_{xy}}{12} - \frac{p_{xyy}}{6} + \frac{u v_{yyy}}{6} + \frac{u_{y} v_{yy}}{3} + \frac{v v_{xyy}}{6} - \frac{2 v_{xy} v_{y}}{3} + \frac{v_{x} v_{yy}}{3} - \frac{v_{xyyy}}{6 Re}
\big)


In [16]:
prn([f.subs(TD).doit().simplify() for f in f2])

0

tau =>


-2*exp(-2*t/Re)*sin(y)*cos(x)/Re**2

h^2 =>


(-3*Re*sin(x) + exp(2*t/Re)*sin(y))*exp(-4*t/Re)*cos(x)/(6*Re)

In [17]:
prn([f.subs(KF).doit().simplify() for f in f2])

0

tau =>


0

h^2 =>


(2*Re**4*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2)*sin(pi*y)**2 - Re**4*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2) + 8*Re**4*exp(x*(Re - sqrt(Re**2 + 16*pi**2)))*sin(pi*y)**4 - 8*Re**4*exp(x*(Re - sqrt(Re**2 + 16*pi**2)))*sin(pi*y)**2 - 6*Re**4*exp(x*(Re - sqrt(Re**2 + 16*pi**2))) - 2*Re**3*sqrt(Re**2 + 16*pi**2)*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2)*sin(pi*y)**2 + Re**3*sqrt(Re**2 + 16*pi**2)*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2) - 8*Re**3*sqrt(Re**2 + 16*pi**2)*exp(x*(Re - sqrt(Re**2 + 16*pi**2)))*sin(pi*y)**4 + 8*Re**3*sqrt(Re**2 + 16*pi**2)*exp(x*(Re - sqrt(Re**2 + 16*pi**2)))*sin(pi*y)**2 + 6*Re**3*sqrt(Re**2 + 16*pi**2)*exp(x*(Re - sqrt(Re**2 + 16*pi**2))) + 16*pi**2*Re**2*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2)*sin(pi*y)**2 - 8*pi**2*Re**2*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2) + 128*pi**2*Re**2*exp(x*(Re - sqrt(Re**2 + 16*pi**2)))*sin(pi*y)**4 - 128*pi**2*Re**2*exp(x*(Re - sqrt(Re**2 + 16*pi**2)))*sin(pi*y)**2 - 72*pi**2*Re**2*exp(x*(Re - sqrt(Re**2 + 16*pi**2))) - 64*pi**2*Re*sqrt(R

In [18]:
prn(F3);print("-"*25)
f3 = NF(F3, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F1, F2, F3, F4], head=False)
prn(f3)
prnlatex(f3)

p_{y} + u*v_{x} + v*v_{y} + v_{t} - v_{xx}/Re - v_{yy}/Re

tau =>


v_{tt}/2

h^2 =>


p_{yyy}/6 + u*v_{xxx}/6 + v*v_{yyy}/6 - v_{xxxx}/(12*Re) - v_{yyyy}/(12*Re)

...

tau =>


v_{tt}/2

h^2 =>


Re**2*p_{y}*u**2/12 + Re**2*u**3*v_{x}/12 + Re**2*u**2*v*v_{y}/12 + Re**2*u**2*v_{t}/12 - Re*p_{ty}/12 + Re*p_{xy}*u/12 + Re*p_{x}*v_{x}/4 - Re*p_{yy}*v/12 + Re*p_{y}*v_{y}/12 - Re*u**2*v_{yy}/12 - Re*u*v_{x}*v_{y}/6 + Re*u_{t}*v_{x}/6 + Re*u_{y}*v*v_{x}/6 - Re*v**2*v_{yy}/12 - Re*v*v_{ty}/6 - Re*v_{tt}/12 + p_{yyy}/3 + u_{y}*v_{xy}/3 + v*v_{yyy}/3 + v_{tyy}/6 + v_{xy}*v_{x}/6 + v_{yy}*v_{y}/2 - v_{yyyy}/(6*Re)

p_{y} + u v_{x} + v v_{y} + v_{t} - \frac{v_{xx}}{Re} - \frac{v_{yy}}{Re}
+\tau\big(
\frac{v_{tt}}{2}
\big)
+h^2\big(
\frac{Re^{2} p_{y} u^{2}}{12} + \frac{Re^{2} u^{3} v_{x}}{12} + \frac{Re^{2} u^{2} v v_{y}}{12} + \frac{Re^{2} u^{2} v_{t}}{12} - \frac{Re p_{ty}}{12} + \frac{Re p_{xy} u}{12} + \frac{Re p_{x} v_{x}}{4} - \frac{Re p_{yy} v}{12} + \frac{Re p_{y} v_{y}}{12} - \frac{Re u^{2} v_{yy}}{12} - \frac{Re u v_{x} v_{y}}{6} + \frac{Re u_{t} v_{x}}{6} + \frac{Re u_{y} v v_{x}}{6} - \frac{Re v^{2} v_{yy}}{12} - \frac{Re v v_{ty}}{6} - \frac{Re v_{tt}}{12} + \frac{p_{yyy}}{3} + \frac{u_{y} v_{xy}}{3} + \frac{v v_{yyy}}{3} + \frac{v_{tyy}}{6} + \frac{v_{xy} v_{x}}{6} + \frac{v_{yy} v_{y}}{2} - \frac{v_{yyyy}}{6 Re}
\big)


In [19]:
prn([f.subs(TD).doit().simplify() for f in f3])

0

tau =>


2*exp(-2*t/Re)*sin(x)*cos(y)/Re**2

h^2 =>


-(3*Re*sin(y) + exp(2*t/Re)*sin(x))*exp(-4*t/Re)*cos(y)/(6*Re)

In [20]:
prn([f.subs(KF).doit().simplify() for f in f3])

0

tau =>


0

h^2 =>


(Re - sqrt(Re**2 + 16*pi**2))*(Re**4*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2) + 4*Re**4*exp(x*(Re - sqrt(Re**2 + 16*pi**2)))*sin(pi*y)**2 - 2*Re**4*exp(x*(Re - sqrt(Re**2 + 16*pi**2))) - Re**3*sqrt(Re**2 + 16*pi**2)*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2) - 4*Re**3*sqrt(Re**2 + 16*pi**2)*exp(x*(Re - sqrt(Re**2 + 16*pi**2)))*sin(pi*y)**2 + 2*Re**3*sqrt(Re**2 + 16*pi**2)*exp(x*(Re - sqrt(Re**2 + 16*pi**2))) + 8*pi**2*Re**2*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2) + 64*pi**2*Re**2*exp(x*(Re - sqrt(Re**2 + 16*pi**2)))*sin(pi*y)**2 - 32*pi**2*Re**2*exp(x*(Re - sqrt(Re**2 + 16*pi**2))) - 32*pi**2*Re*sqrt(Re**2 + 16*pi**2)*exp(x*(Re - sqrt(Re**2 + 16*pi**2)))*sin(pi*y)**2 + 16*pi**2*Re*sqrt(Re**2 + 16*pi**2)*exp(x*(Re - sqrt(Re**2 + 16*pi**2))) - 64*pi**4*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2))*sin(2*pi*y)/(96*pi*Re)

In [21]:
prn(F4);print("-"*25)
f4 = NF(F4, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F1, F2, F3, F4], head=False)
prn(f4)
prnlatex(f4)

p_{xx} + p_{yy} - 2*u_{x}*v_{y} + 2*u_{y}*v_{x}

tau =>


0

h^2 =>


p_{xxxx}/12 + p_{yyyy}/12 - u_{xxx}*v_{y}/3 - u_{x}*v_{yyy}/3 + u_{yyy}*v_{x}/3 + u_{y}*v_{xxx}/3

...

tau =>


0

h^2 =>


Re**2*p_{x}*v*v_{x}/2 + Re**2*p_{y}*u*u_{y}/6 + Re**2*u**2*u_{y}*v_{x}/6 + Re**2*u*u_{y}*v*v_{y}/6 + Re**2*u*u_{y}*v_{t}/6 - Re**2*u*v*v_{x}*v_{y}/2 + Re**2*u_{t}*v*v_{x}/2 + Re**2*u_{y}*v**2*v_{x}/2 + Re*p_{xy}*u_{y}/6 + Re*p_{xy}*v_{x}/2 + Re*p_{x}*v_{xy}/3 + Re*p_{y}*v_{yy}/3 - Re*u*u_{y}*v_{yy}/6 - Re*u*v_{xy}*v_{y}/3 - Re*u*v_{x}*v_{yy}/6 + Re*u_{ty}*v_{x}/2 + Re*u_{t}*v_{xy}/3 + Re*u_{y}*v*v_{xy}/2 + Re*u_{y}*v_{tx}/6 + Re*v*v_{xy}*v_{x}/2 + Re*v*v_{yy}*v_{y}/3 + Re*v_{t}*v_{yy}/3 + p_{yyyy}/6 + 2*v_{xyy}*v_{x}/3 + 2*v_{yyy}*v_{y}/3

p_{xx} + p_{yy} - 2 u_{x} v_{y} + 2 u_{y} v_{x}
+\tau\big(
0
\big)
+h^2\big(
\frac{Re^{2} p_{x} v v_{x}}{2} + \frac{Re^{2} p_{y} u u_{y}}{6} + \frac{Re^{2} u^{2} u_{y} v_{x}}{6} + \frac{Re^{2} u u_{y} v v_{y}}{6} + \frac{Re^{2} u u_{y} v_{t}}{6} - \frac{Re^{2} u v v_{x} v_{y}}{2} + \frac{Re^{2} u_{t} v v_{x}}{2} + \frac{Re^{2} u_{y} v^{2} v_{x}}{2} + \frac{Re p_{xy} u_{y}}{6} + \frac{Re p_{xy} v_{x}}{2} + \frac{Re p_{x} v_{xy}}{3} + \frac{Re p_{y} v_{yy}}{3} - \frac{Re u u_{y} v_{yy}}{6} - \frac{Re u v_{xy} v_{y}}{3} - \frac{Re u v_{x} v_{yy}}{6} + \frac{Re u_{ty} v_{x}}{2} + \frac{Re u_{t} v_{xy}}{3} + \frac{Re u_{y} v v_{xy}}{2} + \frac{Re u_{y} v_{tx}}{6} + \frac{Re v v_{xy} v_{x}}{2} + \frac{Re v v_{yy} v_{y}}{3} + \frac{Re v_{t} v_{yy}}{3} + \frac{p_{yyyy}}{6} + \frac{2 v_{xyy} v_{x}}{3} + \frac{2 v_{yyy} v_{y}}{3}
\big)


In [22]:
prn([f.subs(TD).doit().simplify() for f in f4])

0

tau =>


0

h^2 =>


0

In [23]:
prn([f.subs(KF).doit().simplify() for f in f4])

0

tau =>


0

h^2 =>


(Re - sqrt(Re**2 + 16*pi**2))*(-Re**3 + Re**2*sqrt(Re**2 + 16*pi**2) - 16*pi**2*Re + 8*pi**2*sqrt(Re**2 + 16*pi**2))*exp(x*(Re - sqrt(Re**2 + 16*pi**2)))/12

In [24]:
spoly = [a.diff(x) + b.diff(y) for a, b in zip(F2, F3)]
spoly = NF(spoly, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F1, F2, F3, F4], head=True)
prn(spoly)
prnlatex(spoly)

0

tau =>


0

h^2 =>


Re**2*p_{x}*v*v_{x}/2 - Re**2*p_{y}*u*u_{y}/2 - Re**2*u**2*u_{y}*v_{x}/2 - Re**2*u*u_{y}*v*v_{y}/2 - Re**2*u*u_{y}*v_{t}/2 - Re**2*u*v*v_{x}*v_{y}/2 + Re**2*u_{t}*v*v_{x}/2 + Re**2*u_{y}*v**2*v_{x}/2 - Re*p_{xy}*u_{y}/2 + Re*p_{xy}*v_{x}/2 + 3*Re*p_{x}*v_{xy}/2 - Re*p_{yy}*v_{y}/2 + Re*p_{y}*v_{yy}/2 + Re*u*u_{y}*v_{yy}/2 - 2*Re*u*v_{xy}*v_{y} + Re*u_{ty}*v_{x}/2 + 3*Re*u_{t}*v_{xy}/2 + Re*u_{y}*v*v_{xy} - Re*u_{y}*v_{tx}/2 - Re*u_{y}*v_{x}*v_{y}/2 + Re*v*v_{xy}*v_{x}/2 - Re*v_{ty}*v_{y}/2 + Re*v_{t}*v_{yy}/2 - Re*v_{y}**3/2 + p_{yyyy}/2 + 3*u_{y}*v_{xyy}/2 + v_{xyy}*v_{x}/2 + v_{xy}**2 + 2*v_{yyy}*v_{y} + v_{yy}**2

0
+\tau\big(
0
\big)
+h^2\big(
\frac{Re^{2} p_{x} v v_{x}}{2} - \frac{Re^{2} p_{y} u u_{y}}{2} - \frac{Re^{2} u^{2} u_{y} v_{x}}{2} - \frac{Re^{2} u u_{y} v v_{y}}{2} - \frac{Re^{2} u u_{y} v_{t}}{2} - \frac{Re^{2} u v v_{x} v_{y}}{2} + \frac{Re^{2} u_{t} v v_{x}}{2} + \frac{Re^{2} u_{y} v^{2} v_{x}}{2} - \frac{Re p_{xy} u_{y}}{2} + \frac{Re p_{xy} v_{x}}{2} + \frac{3 Re p_{x} v_{xy}}{2} - \frac{Re p_{yy} v_{y}}{2} + \frac{Re p_{y} v_{yy}}{2} + \frac{Re u u_{y} v_{yy}}{2} - 2 Re u v_{xy} v_{y} + \frac{Re u_{ty} v_{x}}{2} + \frac{3 Re u_{t} v_{xy}}{2} + Re u_{y} v v_{xy} - \frac{Re u_{y} v_{tx}}{2} - \frac{Re u_{y} v_{x} v_{y}}{2} + \frac{Re v v_{xy} v_{x}}{2} - \frac{Re v_{ty} v_{y}}{2} + \frac{Re v_{t} v_{yy}}{2} - \frac{Re v_{y}^{3}}{2} + \frac{p_{yyyy}}{2} + \frac{3 u_{y} v_{xyy}}{2} + \frac{v_{xyy} v_{x}}{2} + v_{xy}^{2} + 2 v_{yyy} v_{y} + v_{yy}^{2}
\big)


In [25]:
prn([f.subs(TD).doit().simplify() for f in spoly])

0

tau =>


0

h^2 =>


(sin(x)**2 + sin(y)**2 - 1)*exp(-4*t/Re)

In [26]:
prn([f.subs(KF).doit().simplify() for f in spoly])

0

tau =>


0

h^2 =>


(Re - sqrt(Re**2 + 16*pi**2))*(-Re**3 + Re**2*sqrt(Re**2 + 16*pi**2) - 12*pi**2*Re + 4*pi**2*sqrt(Re**2 + 16*pi**2))*exp(x*(Re - sqrt(Re**2 + 16*pi**2)))/4

In [27]:
spoly = [a.diff(x) + b.diff(y) for a, b in zip(f2, f3)]
spoly = NF(spoly, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4], head=True)
prn(spoly)
prnlatex(spoly)

0

tau =>


0

h^2 =>


Re**2*p_{x}*v*v_{x}/2 - Re**2*p_{y}*u*u_{y}/2 - Re**2*u**2*u_{y}*v_{x}/2 - Re**2*u*u_{y}*v*v_{y}/2 - Re**2*u*u_{y}*v_{t}/2 - Re**2*u*v*v_{x}*v_{y}/2 + Re**2*u_{t}*v*v_{x}/2 + Re**2*u_{y}*v**2*v_{x}/2 - Re*p_{xy}*u_{y}/2 + Re*p_{xy}*v_{x}/2 + 3*Re*p_{x}*v_{xy}/2 - Re*p_{yy}*v_{y}/2 + Re*p_{y}*v_{yy}/2 + Re*u*u_{y}*v_{yy}/2 - 2*Re*u*v_{xy}*v_{y} + Re*u_{ty}*v_{x}/2 + 3*Re*u_{t}*v_{xy}/2 + Re*u_{y}*v*v_{xy} - Re*u_{y}*v_{tx}/2 - Re*u_{y}*v_{x}*v_{y}/2 + Re*v*v_{xy}*v_{x}/2 - Re*v_{ty}*v_{y}/2 + Re*v_{t}*v_{yy}/2 - Re*v_{y}**3/2 + p_{yyyy}/2 + 3*u_{y}*v_{xyy}/2 + v_{xyy}*v_{x}/2 + v_{xy}**2 + 2*v_{yyy}*v_{y} + v_{yy}**2

0
+\tau\big(
0
\big)
+h^2\big(
\frac{Re^{2} p_{x} v v_{x}}{2} - \frac{Re^{2} p_{y} u u_{y}}{2} - \frac{Re^{2} u^{2} u_{y} v_{x}}{2} - \frac{Re^{2} u u_{y} v v_{y}}{2} - \frac{Re^{2} u u_{y} v_{t}}{2} - \frac{Re^{2} u v v_{x} v_{y}}{2} + \frac{Re^{2} u_{t} v v_{x}}{2} + \frac{Re^{2} u_{y} v^{2} v_{x}}{2} - \frac{Re p_{xy} u_{y}}{2} + \frac{Re p_{xy} v_{x}}{2} + \frac{3 Re p_{x} v_{xy}}{2} - \frac{Re p_{yy} v_{y}}{2} + \frac{Re p_{y} v_{yy}}{2} + \frac{Re u u_{y} v_{yy}}{2} - 2 Re u v_{xy} v_{y} + \frac{Re u_{ty} v_{x}}{2} + \frac{3 Re u_{t} v_{xy}}{2} + Re u_{y} v v_{xy} - \frac{Re u_{y} v_{tx}}{2} - \frac{Re u_{y} v_{x} v_{y}}{2} + \frac{Re v v_{xy} v_{x}}{2} - \frac{Re v_{ty} v_{y}}{2} + \frac{Re v_{t} v_{yy}}{2} - \frac{Re v_{y}^{3}}{2} + \frac{p_{yyyy}}{2} + \frac{3 u_{y} v_{xyy}}{2} + \frac{v_{xyy} v_{x}}{2} + v_{xy}^{2} + 2 v_{yyy} v_{y} + v_{yy}^{2}
\big)


In [28]:
F1 = Dx(u) + Dy(v)
F2 = Dt(u) + u*Dx(u) + v*Dy(u) + Dx(p) - DD(u)/Re
F3 = Dt(v) + u*Dx(v) + v*Dy(v) + Dy(p) - DD(v)/Re
F4 = DD(p) - 2*Dx(u)*Dy(v) + 2*Dx(v)*Dy(u)

s = clip(F1.diff(t) - F2.diff(x) - F3.diff(y) +\
         F4 + (u*F1).diff(x) + (v*F1).diff(y) - DD(F1)/Re)
prn(s)


0

tau =>


-u_{ttx}/2 - v_{tty}/2

h^2 =>


-p_{xxxx}/12 - p_{yyyy}/12 - u*v_{xxxy}/6 + u*v_{xyyy}/6 + u_{txxx}/6 + u_{xxxy}*v/6 - u_{xxx}*v_{y}/6 - u_{xyyy}*v/6 - u_{x}*v_{yyy}/6 + u_{yyy}*v_{x}/6 + u_{y}*v_{xxx}/6 + v_{tyyy}/6 - u_{xxxxx}/(6*Re) - u_{xxxyy}/(6*Re) - v_{xxyyy}/(6*Re) - v_{yyyyy}/(6*Re)

In [29]:
s = NF(s, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4], head=True)
prn(s)
prnlatex(s)

0

tau =>


0

h^2 =>


-Re**2*p_{x}*v*v_{x}/2 + Re**2*p_{y}*u*u_{y}/2 + Re**2*u**2*u_{y}*v_{x}/2 + Re**2*u*u_{y}*v*v_{y}/2 + Re**2*u*u_{y}*v_{t}/2 + Re**2*u*v*v_{x}*v_{y}/2 - Re**2*u_{t}*v*v_{x}/2 - Re**2*u_{y}*v**2*v_{x}/2 + Re*p_{xy}*u_{y}/2 - Re*p_{xy}*v_{x}/2 - 3*Re*p_{x}*v_{xy}/2 + Re*p_{yy}*v_{y}/2 - Re*p_{y}*v_{yy}/2 - Re*u*u_{y}*v_{yy}/2 + 2*Re*u*v_{xy}*v_{y} - Re*u_{ty}*v_{x}/2 - 3*Re*u_{t}*v_{xy}/2 - Re*u_{y}*v*v_{xy} + Re*u_{y}*v_{tx}/2 + Re*u_{y}*v_{x}*v_{y}/2 - Re*v*v_{xy}*v_{x}/2 + Re*v_{ty}*v_{y}/2 - Re*v_{t}*v_{yy}/2 + Re*v_{y}**3/2 - p_{yyyy}/2 - 3*u_{y}*v_{xyy}/2 - v_{xyy}*v_{x}/2 - v_{xy}**2 - 2*v_{yyy}*v_{y} - v_{yy}**2

0
+\tau\big(
0
\big)
+h^2\big(
- \frac{Re^{2} p_{x} v v_{x}}{2} + \frac{Re^{2} p_{y} u u_{y}}{2} + \frac{Re^{2} u^{2} u_{y} v_{x}}{2} + \frac{Re^{2} u u_{y} v v_{y}}{2} + \frac{Re^{2} u u_{y} v_{t}}{2} + \frac{Re^{2} u v v_{x} v_{y}}{2} - \frac{Re^{2} u_{t} v v_{x}}{2} - \frac{Re^{2} u_{y} v^{2} v_{x}}{2} + \frac{Re p_{xy} u_{y}}{2} - \frac{Re p_{xy} v_{x}}{2} - \frac{3 Re p_{x} v_{xy}}{2} + \frac{Re p_{yy} v_{y}}{2} - \frac{Re p_{y} v_{yy}}{2} - \frac{Re u u_{y} v_{yy}}{2} + 2 Re u v_{xy} v_{y} - \frac{Re u_{ty} v_{x}}{2} - \frac{3 Re u_{t} v_{xy}}{2} - Re u_{y} v v_{xy} + \frac{Re u_{y} v_{tx}}{2} + \frac{Re u_{y} v_{x} v_{y}}{2} - \frac{Re v v_{xy} v_{x}}{2} + \frac{Re v_{ty} v_{y}}{2} - \frac{Re v_{t} v_{yy}}{2} + \frac{Re v_{y}^{3}}{2} - \frac{p_{yyyy}}{2} - \frac{3 u_{y} v_{xyy}}{2} - \frac{v_{xyy} v_{x}}{2} - v_{xy}^{2} - 2 v_{yyy} v_{y} - v_{yy}^{2}
\big)


In [30]:
n, j, k = symbols(r'n, j, k', real=True)
U, V, P = (f(n, j, k) for f in symbols('u, v, p', cls=Function))

In [31]:
def St(a):
    return (a.subs({n:n+1}) - a)/(tau)
def Sx(a):
    return (a.subs({j:j+1}) - a.subs({j:j-1}))/(2*h)
def Sy(a):
    return (a.subs({k:k+1}) - a.subs({k:k-1}))/(2*h)

def SS(a):
    return (a.subs({j:j+1}) + a.subs({j:j-1}) +\
                -4*a +\
           a.subs({k:k+1}) - a.subs({k:k-1}))/h**2

In [32]:
S1 = Sx(U) + Sy(V)
S2 = St(U) + U*Sx(U) + V*Sy(U) + Sx(P) - SS(U)/Re
S3 = St(V) + U*Sx(V) + V*Sy(V) + Sy(P) - SS(V)/Re
S4 = SS(P) - 2*Sx(U)*Sy(V) + 2*Sx(V)*Sy(V)

s = St(S1) - Sx(S2) - Sy(S3)  - SS(S1)/Re

In [33]:
simplify(s + Sx(Sx(P)) + Sy(Sy(P)) + Sx(V*Sy(U)) + Sy(U*Sx(V))\
                                   + Sx(U*Sx(U)) + Sy(V*Sy(V)))

0

In [34]:
simplify(S4 - Sx(Sx(P)) + Sy(Sy(P)) + Sx(V*Sy(U)) + Sy(U*Sx(V))\
                                   + Sx(U*Sx(U)) + Sy(V*Sy(V)))

(-(u(n, j, k) - u(n, j - 2, k))*u(n, j - 1, k) - (u(n, j, k) - u(n, j + 2, k))*u(n, j + 1, k) - 2*(u(n, j - 1, k) - u(n, j + 1, k))*(v(n, j, k - 1) - v(n, j, k + 1)) + (u(n, j - 1, k - 1) - u(n, j - 1, k + 1))*v(n, j - 1, k) - (u(n, j + 1, k - 1) - u(n, j + 1, k + 1))*v(n, j + 1, k) - (v(n, j, k) - v(n, j, k - 2))*v(n, j, k - 1) - (v(n, j, k) - v(n, j, k + 2))*v(n, j, k + 1) + 2*(v(n, j, k - 1) - v(n, j, k + 1))*(v(n, j - 1, k) - v(n, j + 1, k)) + (v(n, j - 1, k - 1) - v(n, j + 1, k - 1))*u(n, j, k - 1) - (v(n, j - 1, k + 1) - v(n, j + 1, k + 1))*u(n, j, k + 1) - 16*p(n, j, k) + p(n, j, k - 2) - 4*p(n, j, k - 1) + 4*p(n, j, k + 1) + p(n, j, k + 2) - p(n, j - 2, k) + 4*p(n, j - 1, k) + 4*p(n, j + 1, k) - p(n, j + 2, k))/(4*h**2)